## Import Library

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from modul.gravimetri import gravimetri
from modul.bc_model import bc_model
from modul.hs_model import hs_model
from modul.pm_model import pm_model
from modul.extraterrestrial_radiation import extraterrestrial_radiation

## Inisiasi Variabel

Variabel Bebas

In [11]:
# BC Metode berdasarkan tabel persentasi hari terang
persentase_hari_terang = 0.28 # (okt: 0.28; nov: 0.29)

analisis_hari_ke = 'day 1'

Variabel Kontrol

In [12]:
# Latitude
latitude = -8.18

# Massa jenis air dalam kg/m^3
rho_air = 1000

sisi_media_tanaman = {
    'A1' : 12.5, 'A2' : 12.5, 'A3' : 12.5, 'A4' : 12.5,
    'B1' : 15.0, 'B2' : 15.0, 'B3' : 15.0, 'B4' : 15.0,
    'C1' : 17.5, 'C2' : 17.5, 'C3' : 17.5, 'C4' : 17.5,
    'D1' : 20.0, 'D2' : 20.0, 'D3' : 20.0, 'D4' : 20.0,
    'E1' : 22.5, 'E2' : 22.5, 'E3' : 22.5, 'E4' : 22.5,
    'F1' : 25.0, 'F2' : 25.0, 'F3' : 25.0, 'F4' : 25.0, }  # Satuan cm


## Import Dataset

In [ ]:
# Import file excel
data_primer = 'dataset primer/Dataset_ET.xlsx'
dataset_sekunder = 'dataset sekunder/dataset_sekunder.xlsx'

# Membaca semua sheet dari file Excel
df_data_primer = pd.read_excel(data_primer, sheet_name=analisis_hari_ke)
df_data_sekunder = pd.read_excel(dataset_sekunder, sheet_name=analisis_hari_ke)

TypeError: ExcelFile.__init__() got an unexpected keyword argument 'sheet_name'

In [14]:
df_data_primer

,waktu,berat_A1,supply_air_A1,berat_A2,supply_air_A2,berat_A3,supply_air_A3,berat_A4,supply_air_A4,berat_B1,...,supply_air_F2,berat_F3,supply_air_F3,berat_F4,supply_air_F4,curah_hujan,intensitas_matahari,suhu_udara,kelembaban_udara,et0
0,8,2.036,0.00,2.750,0.00,3.790,0.00,3.127,0.00,NaN,...,0.0,16.871,0.0,17.275,0.0,0,59500,37.0,53,0.00
1,9,2.010,0.00,2.722,0.00,3.773,0.00,3.107,0.00,NaN,...,0.0,16.800,0.0,17.145,0.0,0,90000,40.0,39,0.20
2,10,2.200,0.25,2.936,0.25,3.958,0.25,3.343,0.25,NaN,...,1.0,17.984,1.0,18.005,1.0,0,99000,42.7,33,0.25
3,11,2.100,0.00,2.877,0.00,3.909,0.00,3.209,0.00,NaN,...,0.0,17.317,0.0,17.950,0.0,0,24500,37.9,40,0.25
4,12,2.068,0.00,2.870,0.00,3.900,0.00,3.200,0.00,NaN,...,0.0,17.228,0.0,17.827,0.0,0,10400,40.9,34,0.25
5,13,2.064,0.00,2.842,0.00,3.850,0.00,3.154,0.00,NaN,...,0.0,17.221,0.0,17.807,0.0,0,105000,43.9,31,0.25
6,14,2.053,0.00,2.834,0.00,3.832,0.00,3.127,0.00,NaN,...,0.0,17.150,0.0,17.754,0.0,0,56000,41.7,32,0.35
7,15,2.207,0.25,2.960,0.25,3.944,0.25,3.271,0.50,NaN,...,1.5,18.642,1.5,19.113,1.5,0,42000,40.8,35,0.40


In [15]:
df_data_sekunder

,DY,HR,suhu_udara,kelembapan_relatif,kelebapan_spesifik,curah_hujan,kecepatan_angin
0,22,8,24.94,15.75,75.56,0.01,3.22
1,22,9,27.19,14.95,62.75,0.00,3.14
2,22,10,29.21,14.22,53.19,0.00,3.04
3,22,11,30.81,13.73,46.75,0.00,3.01
4,22,12,31.94,13.49,43.00,0.00,3.02
5,22,13,32.36,13.43,41.75,0.00,3.00
6,22,14,32.22,13.18,41.25,0.00,3.03
7,22,15,31.45,13.18,43.06,0.00,3.07


## Definisi Fungsi

In [16]:
# Mencari luas area di dalam variabel
def hitung_luas_media(sisi):
        cm_to_mm = sisi * 10
        luas = cm_to_mm ** 2
        return luas              # Hasil dalam satuan mm

In [17]:
# Pola Regular Expression
berat_pattern = r'^berat_[A-F][1-4]$'
supply_air_pattern = r'^supply_air_[A-F][1-4]$'

# Filter kolom yang sesuai dengan pola regex
berat_columns = [col for col in df_data_primer.columns if re.match(berat_pattern, col)]
supply_air_columns = [col for col in df_data_primer.columns if re.match(supply_air_pattern, col)]

## ET Model Gravimetri/Penimbangan

In [18]:
for kode in sisi_media_tanaman.keys():
    kode_berat = f'berat_{kode}'
    kode_irigasi = f'supply_air_{kode}'
    
    # Buat array untuk variabel yang akan digunakan
    waktu = df_data_primer['waktu'].array
    berat = df_data_primer[kode_berat].array                            # Satuan kg
    irigasi_liter = df_data_primer[kode_irigasi].fillna(0).array        # Satuan liter
    curah_hujan = df_data_sekunder['curah_hujan'].array                 # Satuan mm

    # Hitung luas alas media tanaman
    luas_media = hitung_luas_media(sisi_media_tanaman[kode])    # Satuan mm^2
    # print(f'luas media {kode}: {luas_media}')

    # Ubah irigasi ke satuan mm
    irigasi_mm = (irigasi_liter * 1e6) / (luas_media)           # Satuan mm
    # print(f'irigasi mm {kode}: {irigasi_mm}')

    # Ubah berat ke satuan volume mm^3
    volume = berat * 1e6                                        # Satuan mm^3
    # print(f'volume {kode}: {volume}')

    # Hitung total supply air
    supply_air = irigasi_mm + curah_hujan                       # Satuan mm
    # print(f'supply air {kode}: {supply_air}')

    perubahan_berat, perubahan_et, et_total = evapotranspirasi = gravimetri(berat = volume,
                                                                            waktu = waktu,
                                                                            luas = luas_media,
                                                                            supply_air = supply_air)

    print(f"ET {kode} : {et_total:.5} mm/8jam")


ET A1 : 2.56 mm/8jam
ET A2 : 3.072 mm/8jam
ET A3 : 2.656 mm/8jam
ET A4 : 6.592 mm/8jam
ET B1 : 0.0 mm/8jam
ET B2 : 0.0 mm/8jam
ET B3 : 0.0 mm/8jam
ET B4 : 0.0 mm/8jam
ET C1 : 3.249 mm/8jam
ET C2 : 7.3959 mm/8jam
ET C3 : 5.2898 mm/8jam
ET C4 : 6.3184 mm/8jam
ET D1 : 2.2625 mm/8jam
ET D2 : 0.6125 mm/8jam
ET D3 : 7.525 mm/8jam
ET D4 : 7.95 mm/8jam
ET E1 : 7.5753 mm/8jam
ET E2 : 8.1481 mm/8jam
ET E3 : 13.067 mm/8jam
ET E4 : 7.6642 mm/8jam
ET F1 : 23.984 mm/8jam
ET F2 : 13.456 mm/8jam
ET F3 : 15.848 mm/8jam
ET F4 : 9.88 mm/8jam
